In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/DL\ Alpha\ GO

/content/drive/My Drive/DL Alpha GO


In [3]:
import numpy as np
%pip install sgfmill
from sgfmill import sgf
import pandas as pd
import os
from tqdm import tqdm

import Go_Board
%tensorflow_version 2.x
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, ZeroPadding2D


from os import listdir
from os.path import isfile, join

TensorFlow 2.x selected.


In [0]:
dataset_dir = './go9-large/'
board_size = 9
proportion_dataset = 45000
training = True
data_preprocessing = False

In [0]:
if data_preprocessing:
    examples = []
    for root, dirs, files in os.walk(dataset_dir):
      for filename in files:
          examples.append(filename)
    print(len(examples),"Games")
    examples = examples[:proportion_dataset]

In [0]:
X = []
Y = []
if data_preprocessing:

    for example in tqdm(examples):
        board_go = Go_Board.Our_Board(board_size=board_size)
        with open(dataset_dir+example, "rb") as f:
            game = sgf.Sgf_game.from_bytes(f.read())
        winner = game.get_winner()
        board_size = game.get_size()
        root_node = game.get_root()

        board = np.zeros(shape=(board_size,board_size))

        for node in game.get_main_sequence():

            if node.get_move()[0] is not None:
                move = node.get_move()
                player = move[0]
                coord = (move[1][1],move[1][0]) if move[1] is not None else (-1,-1)

                board  = board_go.get_matrix()

                y = np.zeros(shape=(board_size,board_size))
                if(coord[0] != -1 and coord[1] != -1):
                    y[coord[0]][coord[1]] = 1
                    y = y.reshape(board_size*board_size)
                    y = np.append(y,[0])
                    y = y.reshape(board_size*board_size+1)
                else:
                    y = np.append(y,[1])
                    y = y.reshape(board_size*board_size+1)

                x = board * -1 if player == 'w' else board
                board_go.push(coord)
                X.append(x)
                Y.append(y)

In [0]:
if data_preprocessing:
    X = np.reshape(X,(len(X),board_size,board_size,1))
    print("X shape = ",X.shape)

    Y = np.reshape(Y,(len(Y),82))
    print("Y shape = ",Y.shape)

In [8]:
if data_preprocessing:
    np.save("dataset_X", X)
    np.save("dataset_Y", Y)
else:
    X = np.load('dataset_X.npy')
    Y = np.load("dataset_Y.npy")
    print("X shape = ",X.shape)
    print("Y shape = ",Y.shape)

X shape =  (2165982, 9, 9, 1)
Y shape =  (2165982, 82)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(1451207, 9, 9, 1) (714775, 9, 9, 1)
(1451207, 82) (714775, 82)


In [10]:
model = models.Sequential()

model.add(ZeroPadding2D(padding=3, input_shape=(9,9,1)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(layers.Flatten())

model.add(layers.Dense(82, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 15, 15, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 13, 13, 32)        0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 32)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 19, 19, 32)        0

In [11]:
if training:
    SGD = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)
    Adam = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=SGD,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=5,batch_size=128)


Train on 1451207 samples, validate on 714775 samples
Epoch 1/5
1451207/1451207 [==============================] - 106s 73us/sample - loss: 3.9183 - accuracy: 0.0589 - val_loss: 3.5889 - val_accuracy: 0.1141
Epoch 2/5
1451207/1451207 [==============================] - 99s 68us/sample - loss: 2.7920 - accuracy: 0.2477 - val_loss: 2.5898 - val_accuracy: 0.2871
Epoch 3/5
1451207/1451207 [==============================] - 101s 70us/sample - loss: 2.4926 - accuracy: 0.3088 - val_loss: 2.4591 - val_accuracy: 0.3159
Epoch 4/5
1451207/1451207 [==============================] - 101s 70us/sample - loss: 2.4174 - accuracy: 0.3257 - val_loss: 2.4123 - val_accuracy: 0.3269
Epoch 5/5
1451207/1451207 [==============================] - 101s 70us/sample - loss: 2.3708 - accuracy: 0.3361 - val_loss: 2.3927 - val_accuracy: 0.3328


In [0]:
if training:
    model.save('models/model_'+str(board_size)+'_'+str(proportion_dataset)+'.h5')

In [0]:
new_model = keras.models.load_model('models/model_'+str(board_size)+'_'+str(proportion_dataset)+'.h5')

In [0]:
pred = new_model.predict(X_test)

In [0]:
def get_coord(pred):
    return pred//board_size,pred%board_size

In [0]:
print(X_test[0].reshape(9,9))
print(get_coord(np.argmax(pred[0])))